**Sommario**

- [Schema e ciclo di vita del database](#schema-e-ciclo-di-vita-del-database)
  - [Creazione delle tabelle nel DB con `CREATE TABLE`](#creazione-delle-tabelle-nel-db-con-create-table)
    - [Constraints (vincoli)](#constraints-vincoli)
    - [Tipi di dato](#tipi-di-dato)
    - [Nomi delle tabelle](#nomi-delle-tabelle)
- [Operazioni CRUD con SQLite3](#operazioni-crud-con-sqlite3)
  - [`db.py`](#dbpy)
  - [`app.py`](#apppy)
  - [C.R.U.D.](#crud)
    - [CREATE con `INSERT INTO`](#create-con-insert-into)
    - [READ con `SELECT ... FROM`](#read-con-select--from)
    - [UPDATE con `UPDATE ... SET`](#update-con-update--set)
    - [DELETE con `DELETE FROM`](#delete-con-delete-from)
  - [Connessione e gestione delle transazioni](#connessione-e-gestione-delle-transazioni)
    - [Connessione con `sqlite3.connect()`](#connessione-con-sqlite3connect)
    - [Transazione](#transazione)
    - [Commit con `Connection.commit()`](#commit-con-connectioncommit)
    - [Rollback con `Connection.rollback()`](#rollback-con-connectionrollback)
    - [Chiusura con `Connection.close()`](#chiusura-con-connectionclose)
    - [`Connection` *context manager* con `with`](#connection-context-manager-con-with)
    - [`with conn:` ACID](#with-conn-acid)
    - [Gestione errori e rollback con `try ... except`](#gestione-errori-e-rollback-con-try--except)

# Schema e ciclo di vita del database

Prima di iniziare ad eseguire le query sui dati, dobbiamo prima creare il database. Per fare questo dobbiamo innanzitutto definire lo *schema del database*.

Agire sullo schema del database significa creare, modificare o eliminare gli oggetti che compongono il database nel suo complesso. Le prime e più importanti cose che possiamo definire sono le *tabelle* e le loro *colonne* (o *campi*).

Le istruzioni per compiere queste azioni sono un sotto-insieme delle istruzioni SQL, dette *Data Definition Language* (DDL). Le principali sono `CREATE`, `ALTER` e `DROP`.

Inizialmente ci limiteremo a creare le tabelle. Per fare ciò abbiamo `CREATE TABLE`.

## Creazione delle tabelle nel DB con `CREATE TABLE`

L'istruzione `CREATE TABLE` in SQL viene utilizzata per creare una nuova tabella in un database. Di seguito, esploriamo la sintassi generale e i vari componenti dell'istruzione `CREATE TABLE` in SQLite.

Per appropondire: ["CREATE TABLE" sulla documentazione ufficiale](https://www.sqlite.org/lang_createtable.html)

Sintassi generale:

```sql
CREATE TABLE [IF NOT EXISTS] nome_tabella (
    nome_colonna1 tipo_dato [CONSTRAINTS],
    nome_colonna2 tipo_dato [CONSTRAINTS],
    ...
);
```

- **`CREATE TABLE`**: Questa è la parola chiave utilizzata per indicare che si desidera creare una nuova tabella.

- **`IF NOT EXISTS`**: (Opzionale) Questa clausola è utilizzata per evitare errori nel caso in cui la tabella esista già. Se la tabella esiste, l'istruzione non creerà una nuova tabella né genererà un errore.

- **nome_tabella**: Il nome della tabella che si desidera creare.

- **nome_colonna**: Il nome di ciascuna colonna all'interno della tabella.

- **tipo_dato**: Specifica il tipo di dati che la colonna può contenere, come `INTEGER`, `TEXT`, `REAL`, `BLOB`, ecc.

- **CONSTRAINTS**: (Opzionale) Vincoli che definiscono le regole per i dati in quella colonna, come `PRIMARY KEY`, `NOT NULL`, `UNIQUE`, `CHECK`, `DEFAULT`, ecc.

### Constraints (vincoli)

I vincoli sono regole applicate alle colonne per limitare il tipo di dati che possono essere memorizzati in esse. Ecco alcuni vincoli comuni:

- **`PRIMARY KEY`**: Identifica in modo univoco ogni record nella tabella.

- **`UNIQUE`**: Assicura che tutti i valori in una colonna siano unici.

- **`NOT NULL`**: Assicura che una colonna non possa avere valori `NULL`.

- **`CHECK`**: Assicura che tutti i valori in una colonna soddisfino una condizione specifica.

- **`DEFAULT`**: Assegna un valore predefinito a una colonna se nessun valore viene specificato. Se omesso viene usato `NULL`.

Ad esempio:

```sql
CREATE TABLE IF NOT EXISTS corsi (
    id INTEGER PRIMARY KEY,
    nome_corso TEXT NOT NULL UNIQUE,
    descrizione TEXT DEFAULT 'N/D',
    durata INTEGER CHECK(durata > 0)
);
```

### Tipi di dato

Per appropondire: ["Datatypes In SQLite" sulla documentazione ufficiale](https://www.sqlite.org/lang_createtable.html)


Quando si crea una tabella in SQLite, ci sono quattro principali tipi di dati che puoi usare per definire le colonne della tabella:

- **`INTEGER`**: Usato per memorizzare numeri interi (positivi o negativi). Es.  `42`, `-3`, `0`

- **`REAL`**: Usato per memorizzare numeri in virgola mobile (numeri decimali). Es. `3.14`, `-2.718`, `0.0`

- **`TEXT`**: Usato per memorizzare stringhe di testo. Es. `'ciao'`, `'SQLite è fantastico!'`, `'123abc'`

- **`BLOB`**: Usato per memorizzare dati binari, come immagini e file in generale.

Inoltre qualunque colonne possono contenere valori `NULL` a meno che non sia specificato diversamente con il vincolo `NOT NULL` o `PRIMARY KEY`.

Ad esempio:

```sql
CREATE TABLE studenti (
    id INTEGER PRIMARY KEY,
    nome TEXT NOT NULL,
    eta INTEGER,
    peso REAL,
    foto BLOB
);
```
Ecco la corrispondenza dei valori tra Python e SQLite:

| Python type | SQLite type |
|-------------|-------------|
| `None  `    | `NULL`      |
| `int`       | `INTEGER`   |
| `float`     | `REAL`      |
| `str`       | `TEXT`      |
| `bytes`     | `BLOB`      |


### Nomi delle tabelle

In SQLite, i nomi di tabella che sono considerati validi (senza l'uso di parentesi o virgolette) dovrebbero essere qualsiasi combinazione alfanumerica che non inizi con una cifra:

- `abc123` - valido
- `123abc` - non valido
- `abc_123` - valido
- `_123abc` - valido
- `abc-abc` - non valido (sembra un'espressione)
- `abc.abc` - non valido (sembra una notazione *database.tabella*)

Racchiudendo i nomi con le virgolette dovresti essere in grado di usare praticamente qualsiasi cosa come nome di una tabella:

- virgolette: `"Questo dovrebbe-essere un_nome.di/tabella+valido!?"`

SQLite supporta anche la delimitazione in stile SQL Server e MySQL:

- parentesi quadre: `[Questo dovrebbe-essere un_nome.di/tabella+valido!?]`

- backtick: \``Questo dovrebbe-essere un_nome.di/tabella+valido!?`\`

# Operazioni CRUD con SQLite3

Come avete appreso nelle lezioni di SQL, l'acronimo [**CRUD**](https://it.wikipedia.org/wiki/CRUD) sta per CREATE, READ, UPDATE e DELETE, ovvero le quattro operazioni fondamentali che possiamo eseguire sui database.

Vediamo come effettuare queste operazioni con l'utilizzo del modulo standard `sqlite3` di Python.

## `db.py`

Immaginiamo di avere una tabella una tabella `'user'`, creata nel file `db.py`:

```python
import sqlite3

def init_db_tables()

    conn = sqlite3.connect('db.sqlite3')  # Apre una connessione al DB

    with conn:  # Apre un contesto sulla connessione
        cursor = conn.cursor() # Crea un cursore per poter eseguire le query

        # Crea la tabella User se non esiste già
        cursor.execute('''CREATE TABLE IF NOT EXISTS user
                    (id INTEGER PRIMARY KEY AUTOINCREMENT,
                    name TEXT NOT NULL,
                    email TEXT UNIQUE NOT NULL,
                    age INTEGER);''')

        conn.commit()  # Salva le modifiche sul DB.

    conn.close()  # Chiude la connessione
```

## `app.py`

Per poter svolgere le operazioni CRUD nel file del "controller" `app.py`, dobbiamo prima creare le tabelle nel database.

Dopodiché abbiamo due scelte:

- Effettuare le operazioni CRUD prima dell'avvio dell'applicazione Flask. Questo può essere utile per effettuare delle operazioni come l'inizializzazione e il popolamento delle tabelle, azioni di migrazione ecc.

- Effettuare le operazioni CRUD all'interno delle funzioni "endpoint" che gestiscono le route, dunque dopo che l'applicazione e il server Flask sono stati avviati.

```python
# app.py

from flask import Flask
from db import init_db_tables

app = Flask(__name__)  # Crea l'istanza di Flask

...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD
...  # <--- ALL'INTERNO DELLE FUNZIONI DI ENDPOINT
...  # <--- CHE GESTISCONO LE ROUTE.

if __name__ == '__main__':
    with app.app_context():
        init_db_tables()  # Crea le tabelle nel database, se necessario

        ...  # <--- QUA PUOI SVOLGERE LE OPERAZIONI CRUD PRIMA
        ...  # <--- DELL'AVVIO DELL'APP FLASK. PUO' ESSERE UTILE 
        ...  # <--- PER INIZIALIZZARE E POPOLARE IL DB.

    app.run()

```

## C.R.U.D.

Vediamo ora come eseguire le quattro classiche operazioni CRUD con SQLite.

### CREATE con `INSERT INTO`

Per creare un nuovo record nel database:

```python
import sqlite3

name = 'Alice'
email = 'alice.smith@example.com'

conn = sqlite3.connect('db.sqlite3')

with conn:
    cursor = conn.cursor()
    cursor.execute('INSERT INTO User (name, email) VALUES (?, ?);', (name, email))
    new_record_id = cursor.lastrowid  # es. 10 (ID del record creato)

# Terminato il blocco `with` viene fatto commit ma la connessione va chiusa
conn.close()
```

### READ con `SELECT ... FROM`

Per leggere i record dal database:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')

with conn:
    cursor = conn.cursor()
    cursor.execute('''
        SELECT * FROM User
        WHERE name = ?;
    ''')
    results = cursor.fetchall()  # Ottiene tutti i record trovati con la query

conn.close()
```

> PER APPROFONDIRE: Vedi il notebook [sqlite3_READ.ipynb](sqlite3_READ.ipynb)


### UPDATE con `UPDATE ... SET`

Per aggiornare un record esistente:

```python
import sqlite3

user_id = 10
name = 'Bob'
email = 'bob.doe@example.com'

conn = sqlite3.connect('db.sqlite3')

with conn:
    cursor = conn.cursor()
    cursor.execute('''
        UPDATE User
        SET name = ?, email = ?
        WHERE id = ?;
    ''', (name, email, user_id))
    conn.commit()
    upd_row_count = cursor.rowcount  # Ottiene il numero di righe aggiornate

conn.close()
```

### DELETE con `DELETE FROM`

Per eliminare un record esistente:

```python
import sqlite3

conn = sqlite3.connect('db.sqlite3')

with conn:
    cursor = conn.cursor()
    cursor.execute('''
        DELETE FROM User
        WHERE id = ?;
    ''', (user_id,))
    conn.commit()
    del_row_count = cursor.rowcount  # Ottiene il numero di righe eliminate

conn.close()
```

## Connessione e gestione delle transazioni

### Connessione con `sqlite3.connect()`

In `sqlite3`, una connessione rappresenta una singola connessione al database. La connessione è il punto di ingresso per eseguire operazioni SQL, dette *transazioni*:

```python
conn = sqlite3.connect('test.db')
```

### Transazione

Fintanto che una connessione è aperta, possono avvenire più transazioni. Per esempio, quanto invochiamo `Cursor.execute()` o `Cursor.executemany()` vengono effettuate delle transazioni.

Tuttavia bisogna ricordare che le transazioni non provocano modifiche permanenti sul database finché non viene eseguito un _**commit**_ sulla connessione.

Per annullare invece le modifiche non ancora committate, si può fare un _**rollback**_.

Al termine di tutte le transazioni che facciamo, bisogna chiudere la connessione per liberare risorse di sistema. 

### Commit con `Connection.commit()`

Conferma tutte le modifiche fatte nel database durante la transazione. Una volta eseguito il commit, le modifiche diventano permanenti.

```python
conn.commit()
```

### Rollback con `Connection.rollback()`

Annulla tutte le modifiche fatte nella transazione corrente, ripristinando lo stato del database al momento dell'ultimo commit.

```python
conn.rollback()
```

### Chiusura con `Connection.close()`

Chiude la connessione corrente e libera tutte le risorse associate alla connessione vengono. Questo include qualsiasi memoria, file, o altre risorse di sistema che sono state allocate per gestire la connessione.

> **ATTENZIONE**: Aprire molte connessioni senza chiuderle, può portare a un esaurimento delle risorse di sistema! Ricordati dunque di chiudere sempre le connessioni quando non servono più.

### `Connection` *context manager* con `with`

Un oggetto `Connection` può essere utilizzato come *context manager* in un blocco `with`, il quale esegue automaticamente il *commit* o il *rollback* delle transazioni aperte quando si esce dal suo corpo.

Se il corpo dell'istruzione `with` termina senza errori (*exceptions*), la transazione viene committata automaticamente.

Se il commit fallisce o se il corpo dell'istruzione `with` solleva un'eccezione (non intercettata), viene eseguito il rollback della transazione e dunque viene annullata.

Il gestore del contesto non chiude implicitamente la connessione. 

Ricapitolando, quando il blocco `with` termina, viene eseguito automaticamente il commit il ma non la chiusura della connessione. Ricordati dunque di chiudere sempre la connessione!

Fare commit manualmente invece non è obbligatorio, ma è considerato una buona pratica e in alcune circostanze può essere utile se dopo vengono svolte altre operazioni.

### `with conn:` ACID

Quando eseguiamo delle transazioni con il database, dobbiamo possibilmente rispettare il paradigma [**ACID**](https://it.wikipedia.org/wiki/ACID) (Atomicità, Consistenza, Isolamento, Durabilità).

Eseguire le transazioni dentro un blocco `with` ci aiuta e semplifica ottenere le proprietà **ACID**.

Infatti `with` garantisce che le operazioni all'interno del blocco `with` siano atomiche. Se una qualsiasi operazione fallisce, un rollback annullerà tutte le operazioni precedenti nella transazione (ATOMICITÀ). Questo fa sì che il database sia sempre in uno stato consistente (CONSISTENZA) e usando blocchi `with` diversi si isolano le modifiche non committate dalle altre transazioni (ISOLAMENTO). Infine, una volta che una transazione è stata committata, le modifiche sono permanenti e sopravvivono a eventuali crash dell'applicazione o del sistema (DURABILITÀ).


### Gestione errori e rollback con `try ... except`

In pratica `with` ci evita di dover gestire le eccezioni manualmente con blocchi `try ... except`.

Tuttavia quanto segue è sempre possibile e utile:

```python
import sqlite3

try:
    conn = sqlite3.connect('test.db')
    cursor = conn.cursor()
    # La seguente istruzione potrebbe provocare un errore sul DB
    cursor.execute('UPDATE User SET email = ? WHERE id = ?', (email, user_id))
    ... # Dopo la transazione, potrebbero verificarsi errori Python
except sqlite3.Error as err:
    conn.rollback()
    print(f'Si è verificato un errore con il DB: {err}')
except Exception as err:
    conn.rollback()
    print(f'Si è verificato un errore Python generico: {err}')
else:
    conn.commit()
```

Chiaramente, se abbiamo bisogno di un controllo sugli errori, sui messaggi e sui comportamenti da intraprendere in base al tipo di eccezione, il costrutto `try ... except` è necessario.